In [1]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [2]:
import pandas as pd
from io import BytesIO
import numpy as np
import os
import shutil
import pprint
import json
from PIL import Image
from PIL.ImageStat import Stat
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import tensorflow as tf
import pathlib
import cv2

# some settings to make it smoothly runnable in Jupyter
os.environ['KMP_DUPLICATE_LIB_OK']='True'
%matplotlib inline
import matplotlib.pyplot as plt

In [3]:
# install required
!pip install opencv-python mtcnn db  jasper glib   2>&1

     |████████████████████████████████| 2.3MB 7.8MB/s 
     |████████████████████████████████| 174kB 51.3MB/s 
     |████████████████████████████████| 71kB 10.5MB/s 
     |████████████████████████████████| 51kB 6.9MB/s 
  Created wheel for db: filename=db-0.1.1-cp37-none-any.whl size=3895 sha256=4aadf77f35367d692f9b88d5e91334000bde7e0a26790d3ddc4e1d2841e92a6e
  Stored in directory: /root/.cache/pip/wheels/30/eb/ba/237fa002d1d1b2e73cedcefd26a9db37c4b72c7e5156ea0501
  Created wheel for glib: filename=glib-1.0.0-cp37-none-any.whl size=2594 sha256=5c091f19a597140eb73557755c0ad754ef2abc46fac7d27461c1d17dca04e111
  Stored in directory: /root/.cache/pip/wheels/a8/21/79/bbc3cc29e5d14dfb72eee89df860316408976c33bc14659a97
  Created wheel for antiorm: filename=antiorm-1.2.1-cp37-none-any.whl size=31679 sha256=70c1976fbc0f6801c39d5e0c5046f69335f95d7cce42f673ac1544027f8b7dff
  Stored in directory: /root/.cache/pip/wheels/b0/91/4d/f8fe808786ff1cda9e7e99e1b1bbda9196ab26786017965313
Successfully built

In [4]:

def eyes_angle(left_eye, right_eye):
    # find the distances between X and Y coordinates of both eyes
    dY = right_eye[1] - left_eye[1]
    dX = right_eye[0] - left_eye[0]
    # compute the angle using trigonometry
    angle = np.degrees(np.arctan2(dY, dX))
    return angle
    
def scaling_factor(left_eye, right_eye, desired_left_eye, desired_right_eye):
    # find the distances between X and Y coordinates of both eyes
    dY = right_eye[1] - left_eye[1]
    dX = right_eye[0] - left_eye[0]
    # find the actual distance between eyes (the hypotenuse)
    dist = np.sqrt((dX ** 2) + (dY ** 2))
    # find the distance between X and Y coordinates in the desired face (which we will have after scaling)
    desired_dY = desired_right_eye[1] - desired_left_eye[1]
    desired_dX = desired_right_eye[0] - desired_left_eye[0]
    # find the  distance between desired eye coordinates (the hypotenuse)
    desired_dist = np.sqrt((desired_dX ** 2) + (desired_dY ** 2))
    
    # compute the ratio between distances, which is the scale factor
    scaling_factor = desired_dist / dist
    return scaling_factor
    

def crop_and_align(image, left_eye, right_eye, desired_image_width, 
                   desired_left_eye_percentage):
    # find angle of the line between the eyes
    angle = eyes_angle(left_eye, right_eye)

    # assuming desired_left_eye_percentage tells where the eyes should be relative to the image size
    # compute its actual place in the resulted image
    desired_left_eye = (desired_left_eye_percentage[0]*desired_image_width, 
                        desired_left_eye_percentage[1]*desired_image_width)
    # similar compute the mirror coordinates for desired_right_eye
    desired_right_eye = ((1.0-desired_left_eye_percentage[0])*desired_image_width, 
                        desired_left_eye_percentage[1]*desired_image_width)

    # find scaling factor based on where we want our eyes to be in the resulted image
    scale = scaling_factor(left_eye, right_eye, desired_left_eye, desired_right_eye)
    
    # find the center point between two eyes, around which we will rotate the image
    eyes_center = ((left_eye[0] + right_eye[0]) // 2, (left_eye[1] + right_eye[1]) // 2)

    # compute the rotation matrix using OpenCV, rotate and scale around the eyes_center
    M = cv2.getRotationMatrix2D(eyes_center, angle, scale)

    # move the current center of the eyes to the desired coordinates, which are
    # mid point horizontally and the desired level vertically
    tX = desired_image_width * 0.5
    tY = desired_left_eye[1]
    M[0, 2] += (tX - eyes_center[0])
    M[1, 2] += (tY - eyes_center[1])

    # by specifying height and width of the final image
    # as our desired_image_width, we insruct warpAffine to cut off the extra pixels
    w = desired_image_width
    h = desired_image_width

    # using OpenCV warpAffine() apply the M transformation, which will also crop the image
    aligned = cv2.warpAffine(image, M, (w, h), flags=cv2.INTER_CUBIC)
    return aligned

from mtcnn import MTCNN
detector = MTCNN()

# detect one face and its eyes coordinates in the given image
def detect_face(image, desired_size=224, desired_left_eye_percentage=(0.35, 0.35)):
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    detection_result = detector.detect_faces(image_rgb)
    left_eye = detection_result[0]['keypoints']['left_eye']
    right_eye = detection_result[0]['keypoints']['right_eye']
    face = crop_and_align(image, left_eye, right_eye, desired_size, desired_left_eye_percentage)
    if face is not None:
        return face
    return None

# loop through frames in the video and detect faces
def detect_and_save_faces(video_path, limit_faces=-1, save_faces=True):
    detector = MTCNN()
    faces = list()
    # add '_face' at the end to differentiate face images
    face_name = os.path.splitext(video_path)[0] + '_face'
    
    cap = cv2.VideoCapture(video_path)
    num_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    for frame_no in range(num_frames):
        # if the given limit is not -1, loop only until the limit
        if limit_faces != -1 and frame_no >= limit_faces:
            break
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_no)
        ret, frame = cap.read()
        # detect faces
        face = detect_face(frame, desired_size=256, desired_left_eye_percentage=(0.35, 0.35))
        if face is not None:
            faces.append(face)
            if save_faces:
                cv2.imwrite(face_name + '_' + str(frame_no) + '.png', face)
    return faces

## Data download and extraction


In [5]:
# paths to target folders
DEEPFAKES='/content/deepfakes/fakes'
REAL='/content/deepfakes/real'

try:
      os.makedirs(DEEPFAKES)
except OSError:
      pass

try:
      os.makedirs(REAL)
except OSError:
      pass



In [6]:
#download dataset
tf.keras.utils.get_file("deepfake.tar.gz","https://lp-prod-resources.s3.amazonaws.com/other/detectingdeepfakes/DeepfakeTIMIT.tar.gz", extract=False)
!tar -xf /root/.keras/datasets/deepfake.tar.gz -C /content/deepfakes/fakes/

226615296/226611200 [==============================] - 5s 0us/step


In [7]:
tf.keras.utils.get_file("VidTIMIT.zip", "https://lp-prod-resources.s3.amazonaws.com/other/detectingdeepfakes/VidTIMIT.zip", extract=True)
!mv /root/.keras/datasets/VidTIMIT/* /content/deepfakes/real/

1358815232/1358810924 [==============================] - 33s 0us/step


In [8]:
%cd /content/deepfakes

/content/deepfakes


In [9]:
# running provided code on real and fake video
real_faces = detect_and_save_faces('./real/fram1/sa2.avi', limit_faces=5)
fake_faces = detect_and_save_faces('./fakes/DeepfakeTIMIT/higher_quality/fram1/sa2-video-fadg0.avi', limit_faces=5)

In [ ]:
# convert the list of faces to the numpy array
# define a function that will read and display given images
def display_images_side_by_side(im1, im2):
    # note that images in OpenCV are in BGR format, 
    # and to plot with matplotlib, we convert them to RGB
    im1 = cv2.cvtColor(im1, cv2.COLOR_BGR2RGB)
    im2 = cv2.cvtColor(im2, cv2.COLOR_BGR2RGB)

    # show the images side by side
    plt.figure(figsize=(8, 4))
    plt.subplot(1, 2, 1)
    plt.imshow(im1)
    plt.title('Before normalization')
    plt.subplot(1, 2, 2)
    plt.title('After normalization')
    plt.imshow(im2)
    plt.show()

# plot real and fake faces side by side
display_images_side_by_side(real_faces[0], fake_faces[0])

In [ ]:
### Detecting features

Error: Unable to start session for kernel Python 3. Select another kernel to launch with.

### Detecting features

In [ ]:
import skimage.metrics
num_hist_bins = 32

# Note that the number of bins we use for the histogram is a parameter of the system
# more bins - more features
def compute_hist(image, num_bins=32):
    hist, bins = np.histogram(image.ravel(), num_bins, [0,255], density=True)
    return hist


def compute_blurred_image(image, kernel_size=3, sigma=0.5):
    return cv2.GaussianBlur(image, (kernel_size, kernel_size), sigma)

def mse(x, y):
    return skimage.metrics.normalized_root_mse(x, y)

def psnr(x, y):
    return skimage.metrics.peak_signal_noise_ratio(x, y, data_range=255)

def ssim(x, y):
    return skimage.metrics.structural_similarity(x, y, multichannel=True, 
                                                 gaussian_weights=True, sigma=1.5, 
                                                 use_sample_covariance=False, data_range=255)

def compute_features(image):
    image_blurred = compute_blurred_image(image)
    im_ssim = ssim(image, image_blurred)
    im_mse = mse(image, image_blurred)
    im_psnr = psnr(image, image_blurred)
    im_hist = compute_hist(image, num_bins=num_hist_bins)
    features = np.concatenate([[im_ssim], [im_mse], [im_psnr], im_hist])
    return features

# as an example, compute features just for one real and one fake faces
real_feature = compute_features(real_faces[0])
fake_feature = compute_features(fake_faces[0])
print("Real feature vector:\n {} \nof length {}".format(real_feature, len(real_feature)))
print("Deepfake feature vector:\n {} \nof length {}".format(fake_feature, len(fake_feature)))

# notice the last values in both case are zeros, 
# that's because these face have not bright pixels with values near 255,
# so, the last bin of the histogram becomes empty 